<a href="https://colab.research.google.com/github/maysis175/text-processing/blob/master/text_processing_task2B_k20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 17.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=86bc2a87446f326ffd89a4c12c8d79da420b3ee568353107fc194fd22a0323a0
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=524ad35d9d9cb58d2dcb7571a4170181b03369d4a15cddd42e0bd823fa56e17a
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [2]:
#nltk使ったことない人は、pip install してから対話環境等で以下のdataset等をダウンロードしてください
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#dataset読み込み
from nltk.corpus import reuters as corpus

In [4]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears  
among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They  
told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And  
lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long -  
run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will  
impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to  
stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at  
10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports 

In [5]:
#全document数
len(corpus.fileids())

10788

In [6]:
#全documentで学習する場合
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:5])
print("num of docs:", len(docs))

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...], ['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...], ['JAPAN', 'TO', 'REVISE', 'LONG', '-', 'TERM', ...], ['THAI', 'TRADE', 'DEFICIT', 'WIDENS', 'IN', 'FIRST', ...], ['INDONESIA', 'SEES', 'CPO', 'PRICE', 'RISING', ...]]
num of docs: 10788


### 前処理

In [7]:
# stopwordsリストの作成
en_stop = nltk.corpus.stopwords.words('english')

en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<",")."]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

In [8]:
from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

# Cleaning
import re

def cleaning_text(text): 
    pattern = "[0-9]+"
    text = re.sub(pattern, '', text)
    return text

def preprocess_word(word, stopwordset):
    word = word.lower()

    if word in [",","."]:
        return None

    if word in stopwordset:
        return None

    word = cleaning_text(word)

    lemma = wn.morphy(word)
    if lemma is None:
        return word
    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [9]:
#before
print(docs[0][:25]) 

#after
print(preprocess_documents(docs)[0][:25])

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']
['asian', 'exporter', 'fear', 'damage', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen']


### LDA

In [10]:
import gensim
from gensim import corpora

#documentを，gensim LDAが読み込めるデータ構造にする

#辞書の作成
dictionary = corpora.Dictionary(preprocess_documents(docs))
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

#Dictionary:gensimにおける辞書クラス
#token2id属性には単語と辞書IDとの対応が格納される

print(dictionary.token2id)

{'': 0, '>.': 1, 'able': 2, 'accounting': 3, 'action': 4, 'advantage': 5, 'allege': 6, 'allow': 7, 'also': 8, 'american': 9, 'among': 10, 'analyst': 11, 'april': 12, 'asia': 13, 'asian': 14, 'ask': 15, 'association': 16, 'australia': 17, 'australian': 18, 'avow': 19, 'await': 20, 'aware': 21, 'barrier': 22, 'beef': 23, 'beyond': 24, 'big': 25, 'block': 26, 'boost': 27, 'broker': 28, 'budget': 29, 'business': 30, 'businessmen': 31, 'button': 32, 'call': 33, 'canberra': 34, 'capel': 35, 'capital': 36, 'centre': 37, 'chairman': 38, 'chief': 39, 'co': 40, 'coal': 41, 'commercial': 42, 'complete': 43, 'concern': 44, 'conflict': 45, 'continue': 46, 'correspondent': 47, 'cost': 48, 'could': 49, 'country': 50, 'curb': 51, 'cut': 52, 'damage': 53, 'day': 54, 'defuse': 55, 'democratic': 56, 'deputy': 57, 'despite': 58, 'deterioration': 59, 'diplomatic': 60, 'director': 61, 'disadvantage': 62, 'dispute': 63, 'domestically': 64, 'due': 65, 'economic': 66, 'economy': 67, 'effort': 68, 'electric': 6

In [11]:
#before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])

#after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

#これを全文書の全文に適用したのがcorpus_

['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u', '.', 's', '.', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
[(10, 1), (13, 1), (14, 1), (31, 1), (49, 1), (53, 2), (66, 1), (79, 1), (82, 1), (83, 1), (90, 1), (110, 1), (114, 2), (136, 1), (151, 1), (192, 1), (203, 1), (204, 1), (250, 1)]


#### 学習

In [12]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

#### パラメータ確認

In [13]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.017*"trade" + 0.009*"" + 0.009*"japan" + 0.008*"would" + 0.008*"government" + 0.007*"last" + 0.007*"market" + 0.006*"country" + 0.006*"."" + 0.006*"growth" + 0.005*"expect" + 0.005*"industry" + 0.005*"official" + 0.005*"economic" + 0.005*"export"')
(1, '0.011*"gulf" + 0.010*"southern" + 0.010*"iran" + 0.008*"attack" + 0.008*"gasoline" + 0.007*"ag" + 0.007*"california" + 0.007*"brown" + 0.007*"state" + 0.006*"iranian" + 0.006*"tire" + 0.006*"shipping" + 0.005*"secondary" + 0.005*"utilicorp" + 0.005*"weinberger"')
(2, '0.028*"" + 0.019*"inc" + 0.013*"corp" + 0.011*"merger" + 0.010*"unit" + 0.009*"acquisition" + 0.009*"complete" + 0.009*"expect" + 0.008*"sale" + 0.007*"sell" + 0.007*"co" + 0.007*"agreement" + 0.007*"earnings" + 0.006*"cash" + 0.006*"revenue"')
(3, '0.026*"dollar" + 0.017*"currency" + 0.016*"rates" + 0.016*"market" + 0.015*"bank" + 0.014*"dealer" + 0.013*"yen" + 0.010*"exchange" + 0.010*"paris" + 0.009*"" + 0.008*"baker" + 0.008*"would" + 0.008*"rate" + 0.007*"banks

In [14]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(0, 0.8947328), (6, 0.017403755), (16, 0.06947736)]
document ID 1:[(7, 0.44240472), (11, 0.36973304), (13, 0.1580321)]
document ID 2:[(0, 0.4263265), (6, 0.17583667), (8, 0.07582105), (9, 0.12566416), (13, 0.18295673)]
document ID 3:[(0, 0.19246653), (9, 0.11540505), (13, 0.67404044)]
document ID 4:[(6, 0.6329946), (10, 0.07709441), (11, 0.020806236), (13, 0.2531018)]
document ID 5:[(0, 0.27180552), (7, 0.1960064), (15, 0.5171406)]
document ID 6:[(0, 0.41775066), (3, 0.018599428), (6, 0.14307794), (8, 0.07555727), (10, 0.18383996), (13, 0.062208664), (14, 0.09347969)]
document ID 7:[(2, 0.052795134), (4, 0.23901594), (11, 0.20029278), (17, 0.45788684)]
document ID 8:[(5, 0.4352827), (6, 0.28967673), (13, 0.2169583), (17, 0.03522145)]
document ID 9:[(0, 0.11805416), (2, 0.09793922), (3, 0.033536687), (5, 0.053183217), (6, 0.6319762), (9, 0.02401318), (17, 0.031662565)]


In [15]:
#documentのcategory
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [16]:
n=0

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
print(" ".join(docs[n]))

[(0, 0.89470947), (6, 0.01743083), (16, 0.06947824)]
['trade']
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electron

#### 可視化

In [17]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [18]:
#上で可視化したモデルをgoogle drive上にsaveできる

pyLDAvis.save_html(lda_display,'vis.html')

In [19]:
!ls

sample_data  vis.html
